In [1]:
import requests
import json

# Replace these with your values
your_project_id = 'numeric-ion-393603'
your_location_id = 'us-central1'
your_service_id = 'your_service_id'
your_api_key = 'your_api_key'

url = f"https://healthcare.googleapis.com/v1beta1/projects/{your_project_id}/locations/{your_location_id}/services/nlp:analyzeEntities"


In [2]:
from google.oauth2 import service_account
import google.auth.transport.requests
import requests
import json

# Path to your service account key file
KEY_FILE = '/content/numeric-ion-393603-0d0575605119.json'

# Load the credentials from the service account key file
creds = service_account.Credentials.from_service_account_file(
    KEY_FILE,
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

# Create an HTTP authorized client using the credentials
auth_request = google.auth.transport.requests.Request()
creds.refresh(auth_request)



In [3]:
# Add the token to the headers
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': 'Bearer {}'.format(creds.token)
}
text = 'OBSERVED CASE OF SQUAMOUS CELL CARCINOMA RIGHT dorsum foot x 31 may 1990 with WIDE EXCISION with SPLIT SKIN GRAFT with GROUND final HISTOPATHOLOGY REPORT - well differentiated SQUAMOUS CELL CARCINOMA , cut margin - free ALL 8 groin node free of metastatic disease by DOCTOR s ARTERIA PRADA precursor lesion - chronic lichenified dermatitis NOW pt complain ulceroproliferative growth at right side thigh x 4 MONTHS after itching outside BIOPSY - MODERATELY DIFFERENTIATED SQUAMOUS CELL CARCINOMA LESION EXCISION - 5x 3 cm ulceroproliferative growth at right thigh right inguinal scar present palpable node firm , APPROX 1.5 cm present advice- c CHEST RADIOGRAPH LIVER X RECEPTOR MAGNETIC RESONANCE IMAGING slide block review routine blood dressing'
# Replace 'your_text' with the text you want to analyze
data = {
    'documentContent': text
}

# Specify the API endpoint URL
url = url

# Make a POST request to the API endpoint
response = requests.post(url, headers=headers, data=json.dumps(data))

# Get JSON response
response_json = response.json()

# Print the response
print(response_json)


{'entityMentions': [{'mentionId': '1', 'type': 'ANATOMICAL_STRUCTURE', 'text': {'content': 'SQUAMOUS CELL', 'beginOffset': 17}, 'linkedEntities': [{'entityId': 'UMLS/C0221910'}], 'confidence': 0.37108901143074036}, {'mentionId': '2', 'type': 'PROBLEM', 'text': {'content': 'CARCINOMA', 'beginOffset': 31}, 'linkedEntities': [{'entityId': 'UMLS/C0007097'}], 'temporalAssessment': {'value': 'CURRENT', 'confidence': 0.7266263365745544}, 'certaintyAssessment': {'value': 'LIKELY', 'confidence': 0.8828955888748169}, 'subject': {'value': 'PATIENT', 'confidence': 0.9994209408760071}, 'confidence': 0.6681205034255981}, {'mentionId': '3', 'type': 'ANATOMICAL_STRUCTURE', 'text': {'content': 'foot', 'beginOffset': 54}, 'linkedEntities': [{'entityId': 'UMLS/C0016504'}, {'entityId': 'UMLS/C4299097'}], 'confidence': 0.8317325711250305}, {'mentionId': '4', 'type': 'ANATOMICAL_STRUCTURE', 'text': {'content': '1990', 'beginOffset': 68}, 'confidence': 0.3147028684616089}, {'mentionId': '5', 'type': 'PROCEDU

In [4]:
x = response_json

In [12]:
def get_data(x):

    data = []

    if 'entityMentions' in x:
        for entity in x['entityMentions']:
            id_val = entity['mentionId']
            entity_type_val = entity['type']
            text_val = entity['text']['content']
            conf = entity['confidence']

            linked_entity_id = []
            if 'entityMentions' in x:
              for entity in x['entityMentions']:
                if 'linkedEntities' in entity:
                  id = entity['mentionId']
                  text = entity['text']['content']
                  linked_id = entity['linkedEntities']
                  for ids in linked_id:
                    linked_entity_id.append([id,text,ids['entityId']])
                  linked_entities = []
                  if len(linked_entity_id) != 0:
                    for ids in linked_entity_id:
                      id = ids[0]
                      entity_id = ids[2]
                      text = ids[1]
                      for entities in x['entities']:
                        if entities['entityId'] == entity_id:
                          term = entities['preferredTerm']
                          linked_entities.append([id,text,term,entity_id])

            related_entities = []
            if 'relationships' in x:
              for relations in x['relationships']:
                related_entities.append([relations['subjectId'],relations['objectId']])
              relationship = []
              if len(related_entities) != 0:
                relationship = []
                for ids in related_entities:
                    id1 = ids[0]
                    id2 = ids[1]
                    text1 = None  # Initialize text1
                    text2 = None  # Initialize text2
                    for entity in x['entityMentions']:
                        if entity['mentionId'] == str(id1):
                            text1 = entity['text']['content']
                        if entity['mentionId'] == str(id2):
                            text2 = entity['text']['content']

                    relationship.append([id1, text1, id2, text2])

            data.append({
                'id': id_val,
                'entity_type': entity_type_val,
                'text': text_val,
                'confidence':conf,
                'linked from id': id,
                'linked from text': text,
                'linked to id': entity_id,
                'linked to term': term,
                'related from id': id1,
                'related from': text1,
                'related to id': id2,
                'related to term': text2
                })


    return data



In [14]:
import pandas as pd

In [13]:
print(get_data(x))

[{'id': '1', 'entity_type': 'ANATOMICAL_STRUCTURE', 'text': 'SQUAMOUS CELL', 'confidence': 0.37108901143074036, 'linked from id': '31', 'linked from text': 'blood dressing', 'linked to id': 'UMLS/C1705365', 'linked to term': 'Dressing Dosage Form', 'related from id': '9', 'related from': 'CARCINOMA', 'related to id': '8', 'related to term': 'SQUAMOUS CELL'}, {'id': '2', 'entity_type': 'PROBLEM', 'text': 'CARCINOMA', 'confidence': 0.6681205034255981, 'linked from id': '31', 'linked from text': 'blood dressing', 'linked to id': 'UMLS/C1705365', 'linked to term': 'Dressing Dosage Form', 'related from id': '9', 'related from': 'CARCINOMA', 'related to id': '8', 'related to term': 'SQUAMOUS CELL'}, {'id': '3', 'entity_type': 'ANATOMICAL_STRUCTURE', 'text': 'foot', 'confidence': 0.8317325711250305, 'linked from id': '31', 'linked from text': 'blood dressing', 'linked to id': 'UMLS/C1705365', 'linked to term': 'Dressing Dosage Form', 'related from id': '9', 'related from': 'CARCINOMA', 'relat

In [15]:
df1 = pd.DataFrame(get_data(x))

In [17]:
df1.to_csv('sample.csv')

In [ ]:
import ast

In [ ]:
# Convert the string representation of dictionary to actual dictionary in 'NER' column.
df['NER'] = df['NER'].apply(ast.literal_eval)

In [ ]:
df['entities'] = df['NER'].apply(lambda x: get_data(x))
# Explode the list into separate rows
df = df.explode('entities')

# Convert dictionaries into separate columns
df = pd.concat([df.drop(['entities'], axis=1), df['entities'].apply(pd.Series)], axis=1)